In [1]:
import PyPDF2
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
from transcript_functions import *

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rawin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rawin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
filename = '2019_proxy.pdf' 

pdfFileObj = open(filename,'rb')

pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

text = []
for i in range(1, pdfReader.numPages):
    pageObj = pdfReader.getPage(i)
    page = pageObj.extractText()
    page = page.replace('\n \n',', ')
    page = page.replace('\n','')
    page = page.replace('\d+', '')
    text.append(page)

In [3]:
len(text)

110

In [4]:
df = pd.DataFrame(text, columns=['raw_text'])

In [5]:
#tokenize
df["tokens"] = df["raw_text"].map(nltk.word_tokenize)

#make lowercase
df["tokens_clean"] = df["tokens"].map(lambda x: [word.lower() for word in x])

#remove stopwords
stop_words = set(stopwords.words('english'))
df["tokens_clean"] = df["tokens_clean"].map(lambda x: [word for word in x if word not in stop_words])

#remove punctuation
punctuation = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~,."
df["tokens_clean"] = df["tokens_clean"].map(lambda x: [word for word in x if word not in punctuation])

Lemmatize with POS tagging:

In [6]:
# tag Part of Speech
df["tokens_pos"] = df["tokens_clean"].map(nltk.pos_tag)
# filter words with pos not in pos_keep
pos_keep = ["NN", "NNS","VB", "VBD", "VBN", "VBP", "VBZ"] #take out "JJ", "JJR", "JJS"
df["tokens_pos"] = df["tokens_pos"].map(lambda x: [tup for tup in x if tup[1] in pos_keep])

# lemmatize with POS
lemmatizer = WordNetLemmatizer()
df["tokens_pos"] = df["tokens_pos"].map(lambda x: [lemmatizer.lemmatize(tup[0], get_wordnet_pos(tup[1])) for tup in x ])

df["corpus"] = df["tokens_pos"].map(lambda x: ' '.join(x))

#remove numbers
df['corpus'] = df['corpus'].str.replace('\d+', '')

#tokenize
df["tokens_final"] = df["corpus"].map(nltk.word_tokenize)

Lemmatize without POS tagging:

In [7]:
# def lemmatize_text(text):
#     lemmatizer = WordNetLemmatizer()
#     return [lemmatizer.lemmatize(w) for w in text] 

# df["tokens_clean"] = df["tokens_clean"].apply(lemmatize_text)

# df["corpus"] = df["tokens_clean"].map(lambda x: ' '.join(x))

# #remove numbers
# df['corpus'] = df['corpus'].str.replace('\d+', '')

# #tokenize
# df["tokens_final"] = df["corpus"].map(nltk.word_tokenize)

Further Pre-processing

In [8]:
# remove_words = ["director","meeting","committee","mr.","proxy","unit","total","plan","u.s.",
#                 "ceo","walton","stock","vote","walmar˜"]
remove_words =['shareholders™','mr.','director','meeting','plan','committee','stock','company™s','ms.']

df["tokens_final"] = df["tokens_final"].map(lambda x: [word for word in x if word.lower() not in remove_words])

In [9]:
df.head()

,raw_text,tokens,tokens_clean,tokens_pos,corpus,tokens_final
0,At Walmar˜ we save people money so they can li...,"[At, Walmar˜, we, save, people, money, so, the...","[walmar˜, save, people, money, live, better.th...","[walmar˜, save, people, money, live, better.th...",walmar˜ save people money live better.the defi...,"[walmar˜, save, people, money, live, better.th..."
1,3Messages from our Chairman and our Lead Indep...,"[3Messages, from, our, Chairman, and, our, Lea...","[3messages, chairman, lead, independent, direc...","[3messages, chairman, lead, directorwe, please...",messages chairman lead directorwe please atten...,"[messages, chairman, lead, directorwe, please,..."
2,4Fiscal 2019 HighlightsStrategy and Per˚ormanc...,"[4Fiscal, 2019, HighlightsStrategy, and, Per˚o...","[4fiscal, 2019, highlightsstrategy, per˚ormanc...","[highlightsstrategy, per˚ormancewalmar˜, deliv...",highlightsstrategy per˚ormancewalmar˜ deliver ...,"[highlightsstrategy, per˚ormancewalmar˜, deliv..."
3,5$514.4 billionFiscal 2019 Total Revenues$16 b...,"[5, $, 514.4, billionFiscal, 2019, Total, Reve...","[5, 514.4, billionfiscal, 2019, total, revenue...","[total, revenue, billioninvestment, flipkar˜, ...",total revenue billioninvestment flipkar˜ ecomm...,"[total, revenue, billioninvestment, flipkar˜, ..."
4,6Notice of 2019 Annual Shareholders' MeetingHo...,"[6Notice, of, 2019, Annual, Shareholders, ', M...","[6notice, 2019, annual, shareholders, meetingh...","[shareholder, meetinghow, shareholders™, meeti...",shareholder meetinghow shareholders™ meeting˛w...,"[shareholder, meetinghow, meeting˛wednesday, j..."


In [10]:
# count_vectorizer = CountVectorizer()
# count_data = count_vectorizer.fit_transform(df["corpus"])

In [11]:
# number_topics = 15
# number_words = 10

# # fix random_state at 33
# lda = LDA(n_components=number_topics, n_jobs=-1, random_state =33)
# lda.fit(count_data)
       
# print_topics(lda, count_vectorizer, number_words)

Topic 0: Stock value  
Topics 2, 3, 6: Board/Management  
Topic 8: ecommerce  
Topic 10:  
Topic 11: Performance incentives  
Topic 14: Compensation

In [12]:
import gensim
from gensim.utils import simple_preprocess
from gensim.test.utils import common_corpus, common_dictionary

import pprint
pp = pprint.PrettyPrinter(indent=4)

In [13]:
dictionary = gensim.corpora.Dictionary(df["tokens_final"])

dictionary.filter_extremes(no_below=1, no_above=0.5) #, keep_n=100000)

In [14]:
bow_corpus = [dictionary.doc2bow(word) for word in df["tokens_final"]]

In [15]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, 
                                       passes=2, workers=2, chunksize=100, random_state=1000)

In [16]:
pp.pprint(lda_model.print_topics())

[   (   0,
        '0.024*"management" + 0.023*"member" + 0.019*"risk" + 0.018*"review" + '
        '0.014*"audit" + 0.010*"information" + 0.010*"technology" + '
        '0.009*"ecommerce" + 0.008*"policy" + 0.007*"governance"'),
    (   1,
        '0.029*"vote" + 0.014*"shareholder" + 0.012*"accountant" + '
        '0.011*"proposal" + 0.011*"audit" + 0.010*"per˚ormance" + '
        '0.008*"result" + 0.007*"instruction" + 0.007*"hold" + '
        '0.007*"provide"'),
    (   2,
        '0.019*"cmdc" + 0.017*"per˚ormance" + 0.017*"goal" + 0.015*"executive" '
        '+ 0.015*"program" + 0.015*"officer" + 0.013*"pay" + '
        '0.012*"shareholder" + 0.008*"equity" + 0.007*"award"'),
    (   3,
        '0.018*"shareholder" + 0.013*"material" + 0.012*"walmar˜" + '
        '0.010*"vote" + 0.009*"relationship" + 0.009*"business" + '
        '0.008*"receive" + 0.008*"member" + 0.008*"proposal" + 0.007*"proxy"'),
    (   4,
        '0.017*"sale" + 0.016*"calculate" + 0.013*"measure" + 0.013*"

In [17]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

In [18]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.089313  0.092138       1        1  18.633518
3      0.002982 -0.058802       2        1  15.510161
2     -0.008688  0.054775       3        1  12.727200
1     -0.033885 -0.106140       4        1   9.039844
5      0.025109  0.035413       5        1   9.000638
8     -0.027663 -0.048078       6        1   8.202455
4     -0.087738 -0.009015       7        1   8.031013
0      0.118358  0.012124       8        1   7.637776
6      0.063082 -0.007999       9        1   6.893066
7      0.037755  0.035583      10        1   4.324332, topic_info=             Term        Freq       Total Category  logprob  loglift
423          vote  101.000000  101.000000  Default  30.0000  30.0000
429         audit  105.000000  105.000000  Default  29.0000  29.0000
162    management   90.000000   90.000000  Default  28.0000  28.0000
166        member  102.000000  102.000000  Default  27.0000  27.0000
1673  per˜ormance  108.000000  108.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
405     president    3.127477   37.259155  Topic10  -5.4578   0.6632
520       payment    3.354927   53.993298  Topic10  -5.3876   0.3625
549           ceo    3.145736   66.370323  Topic10  -5.4520   0.0917
429         audit    3.367260  105.392242  Topic10  -5.3840  -0.3027
127        equity    2.773770   76.618645  Topic10  -5.5778  -0.1777

[707 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
65        1  0.275204            .
65        2  0.100074            .
65        3  0.125093            .
65        4  0.125093            .
65        5  0.100074            .
...     ...       ...          ...
1212      8  0.604958       ﬁaudit
2807      4  0.403216      ﬁbroker
2823      2  0.781089  ﬁinvestorsﬂ
1502      8  0.299924     ﬁrelated
1502      9  0.299924     ﬁrelated

[2086 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 4, 3, 2, 6, 9, 5, 1, 7, 8])